# Access Azure resources from an online endpoint with a managed identity
## Define configuration for deployment

### System-assigned managed identity

In [ ]:
from azure.ai.ml.entities import ( 
    ManagedOnlineDeployment, ManagedOnlineEndpoint, 
    Model, CodeConfiguration, Environment ) 

In [ ]:
sai_endpoint_name = "my-sai-endpoint"
sai_endpoint = ManagedOnlineEndpoint(
    name=sai_endpoint_name, 
    auth_mode="key"
)

In [ ]:
mod = ManagedOnlineDeployment(name="foo",endpoint_name="bar")
mod.environment_variables['foo'] = 'bar'

In [ ]:
sai_deployment = ManagedOnlineDeployment(
    name="blue", 
    endpoint_name=sai_endpoint_name, 
    model=Model(
        path="../../model-1/model/"
    ),
    code_configuration=CodeConfiguration(
        code="../../model-1/onlinescoring/",
        scoring_script="score_managedidentity.py"
    ), 
    environment=Environment(
        conda_file="../../model-1/environment/conda.yml",
        image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210727.v1"
    ), 
    instance_type="Standard_DS2_v2",
    instance_count=1,
    environment_variables={
        "STORAGE_ACCOUNT_NAME" : "storage_place_holder",
        "STORAGE_CONTAINER_NAME" : "container_place_holder", 
        "FILE_NAME" : "file_place_holder"
    }
)

### User-assigned managed identity

In [ ]:
from azure.ai.ml.entities import ( 
    ManagedOnlineDeployment, ManagedOnlineEndpoint, 
    Model, CodeConfiguration, Environment ) 

In [ ]:
uai_endpoint_name = "my-uai-endpoint"
uai_endpoint = ManagedOnlineEndpoint(
    name=uai_endpoint_name, 
    auth_mode="key", 
    # We will update this value after creating an identity
    identity = None
)

In [ ]:
uai_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=uai_endpoint_name, 
    model=Model(
        path="../../model-1/model/"
    ),
    code_configuration=CodeConfiguration(
        code="../../model-1/onlinescoring/",
        scoring_script="score_managedidentity.py"
    ), 
    environment=Environment(
        conda_file="../../model-1/environment/conda.yml",
        image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210727.v1"
    ), 
    instance_type="Standard_DS2_v2",
    instance_count=1,
    environment_variables={
        "STORAGE_ACCOUNT_NAME" : "storage_place_holder",
        "STORAGE_CONTAINER_NAME" : "container_place_holder", 
        "FILE_NAME" : "file_place_holder", 
        "UAI_CLIENT_ID" : "uai_client_id_place_holder"
    }
)

## Configure variables for deployment

### System-assigned managed identity

In [ ]:
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace_name = "<AML_WORKSPACE_NAME>"

location = "<WORKSPACE_LOCATION>"

storage_account_name = "<STORAGE_ACCOUNT_NAME>"
storage_container_name = "<CONTAINER_TO_ACCESS>"
file_name = "<FILE_TO_ACCESS>"

### User-assigned managed identity

In [ ]:
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace_name = "<AML_WORKSPACE_NAME>"

location = "<WORKSPACE_LOCATION>"

storage_account_name = "<STORAGE_ACCOUNT_NAME>"
storage_container_name = "<CONTAINER_TO_ACCESS>"
file_name = "<FILE_TO_ACCESS>"

uai_name = "<USER_ASSIGNED_IDENTITY_NAME>"

## Create the managed identity

### System-assigned managed identity

With a system-assigned managed identity, Azure creates a managed identity on your behalf.

### User-assigned managed identity

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.msi import ManagedServiceIdentityClient
from azure.mgmt.msi.models import Identity

In [ ]:
credential = DefaultAzureCredential()
msi_client = ManagedServiceIdentityClient(
    subscription_id=subscription_id, 
    credential=credential,)

In [ ]:
msi_client.user_assigned_identities.create_or_update(
    resource_group_name=resource_group,
    resource_name=uai_name,
    parameters = Identity(location=location)
)

In [ ]:
uai_identity = msi_client.user_assigned_identities.get(
    resource_group_name=resource_group, 
    resource_name=uai_name,
)

## Create storage account and container

In [ ]:
from azure.identity import DefaultAzureCredential, AzureCliCredential 
from azure.mgmt.storage import StorageManagementClient
from azure.storage.blob import ContainerClient
from azure.mgmt.storage.models import Sku, StorageAccountCreateParameters, BlobContainer

In [ ]:
credential = AzureCliCredential()
storage_client = StorageManagementClient(credential, subscription_id,)

In [ ]:
storage_account_parameters = StorageAccountCreateParameters(
    sku=Sku(name="Standard_LRS"), 
    kind="Storage", 
    location=location
)

storage_client.storage_accounts.begin_create(
    resource_group_name=resource_group,
    account_name=storage_account_name, 
    parameters=storage_account_parameters)

In [ ]:
storage_account = next((
    account for account in 
    storage_client.storage_accounts.list_by_resource_group(resource_group)
    if account.name==storage_account_name
))

In [ ]:
blob_container = storage_client.blob_containers.create(
    resource_group_name=resource_group, 
    account_name=storage_account_name,
    container_name=storage_container_name,
    blob_container=BlobContainer()
)

In [ ]:
res = storage_client.storage_accounts.list_keys(
    resource_group_name=resource_group, 
    account_name=storage_account_name,
)
key = res.keys[0].value

In [ ]:
container_client = ContainerClient(
    account_url=storage_account.primary_endpoints.blob, 
    container_name=storage_container_name,
    credential=key
)

In [ ]:
file_path = "endpoints/online/managed/managed-identities/hello.txt" 
with open(file_path, "rb") as f:
    container_client.upload_blob(
        name=file_name, 
        data=f.read()
    )

## Create an online endpoint

### System-assigned managed identity

In [ ]:
from azure.ai.ml import MLClient
ml_client = MLClient(credential, subscription_id, resource_group, workspace_name)

In [ ]:
sai_endpoint = ml_client.online_endpoints.begin_create_or_update(sai_endpoint)

In [ ]:
sai_endpoint.identity.as_dict()

### User-assigned managed identity

In [ ]:
from azure.ai.ml import MLClient
ml_client = MLClient(credential, subscription_id, resource_group, workspace_name)

In [ ]:
# Create the endpoint
uai_endpoint = ml_client.online_endpoints.begin_create_or_update(uai_endpoint)

In [ ]:
# Update its identity 
uai_endpoint.identity = uai_endpoint.identity.from_dict({
    "type" : "UserAssigned",
    "user_assigned_identities" : {uai_identity.id : {}}
})

uai_endpoint = ml_client.online_endpoints.begin_create_or_update(uai_endpoint)


In [ ]:
uai_endpoint = ml_client.online_endpoints.get(uai_endpoint_name)

### User-assigned managed identity

In [ ]:
uai_endpoint.identity.as_dict()

## Give access permission to the managed identity

### System-assigned managed identity

In [ ]:
from azure.mgmt.authorization import AuthorizationManagementClient
from azure.mgmt.authorization.models import RoleAssignment, RoleDefinition, RoleAssignmentCreateParameters, RoleAssignmentProperties, RoleAssignmentPropertiesWithScope
import uuid

auth_client = AuthorizationManagementClient(
    credential=credential, 
    subscription_id=subscription_id
)

In [ ]:
sai_endpoint = ml_client.online_endpoints.get(sai_endpoint_name)
system_principal_id = sai_endpoint.identity.principal_id

In [ ]:
workspace = ml_client.workspaces.get(workspace_name)
storage_account = workspace.storage_account

In [ ]:
role_name = "Storage Blob Data Reader"
scope = storage_account 

role_defs = auth_client.role_definitions.list(scope=scope)
role_def = next((r for r in role_defs if r.role_name == role_name))

auth_client.role_assignments.create(
    scope=scope, 
    role_assignment_name=uuid.uuid4(),
    parameters=RoleAssignmentProperties(
        role_definition_id = role_def.id, 
        principal_id = system_principal_id
    )
)

In [ ]:
auth_client.role_assignments.create(
    scope=storage_account, 
    role_assignment_name=uuid.uuid4(),
    parameters=RoleAssignmentProperties(
        role_definition_id = role_def.id, 
        principal_id = system_principal_id
    )
)

### User-assigned managed identity

In [ ]:
from azure.mgmt.authorization.models import RoleAssignment, RoleDefinition, RoleAssignmentProperties

In [ ]:
from azure.mgmt.authorization import AuthorizationManagementClient

auth_client = AuthorizationManagementClient(
    credential=credential, 
    subscription_id=subscription_id
)

In [ ]:
uai_identity = msi_client.user_assigned_identities.get(
    resource_group_name=resource_group, 
    resource_name=uai_name
)
uai_principal_id = uai_identity.principal_id
uai_client_id = uai_identity.client_id

In [ ]:
workspace = ml_client.workspaces.get(workspace_name)
storage_account = workspace.storage_account
container_registry = workspace.container_registry

In [ ]:
role_name = "AcrPull"
scope = container_registry

role_defs = auth_client.role_definitions.list(scope=scope)
role_def = next((r for r in role_defs if r.role_name == role_name))

auth_client.role_assignments.create(
    scope=scope, 
    role_assignment_name=uuid.uuid4(),
    parameters=RoleAssignmentProperties(
        role_definition_id = role_def.id, 
        principal_id = uai_principal_id
    )
)

In [ ]:
role_name = "Storage Blob Data Reader"
scope = storage_account 

role_defs = auth_client.role_definitions.list(scope=scope)
role_def = next((r for r in role_defs if r.role_name == role_name))

auth_client.role_assignments.create(
    scope=scope, 
    role_assignment_name=uuid.uuid4(),
    parameters=RoleAssignmentProperties(
        role_definition_id = role_def.id, 
        principal_id = uai_principal_id
    )
)

## Create a deployment with your configuration

### System-assigned managed identity

In [ ]:
# Set the placeholder environment variables
env_map = {"STORAGE_ACCOUNT_NAME" : storage_account_name, 
"STORAGE_CONTAINER_NAME" : storage_container_name, 
"FILE_NAME" : file_name }

for k, v in env_map.items():
    sai_deployment.environment_variables[k] = v
    

In [ ]:
sai_deployment = ml_client.online_deployments.begin_create_or_update(sai_deployment)

### User-assigned managed identity

In [ ]:
env_map = {"STORAGE_ACCOUNT_NAME" : storage_account_name, 
"STORAGE_CONTAINER_NAME" : storage_container_name, 
"FILE_NAME" : file_name,
"UAI_CLIENT_ID" : uai_client_id}

for k, v in env_map.items():
    uai_deployment.environment_variables[k] = v


In [ ]:
uai_deployment = ml_client.online_deployments.begin_create_or_update(uai_deployment)

## Confirm your endpoint deployed successfully

### System-assigned managed identity

In [ ]:
sample_data = "endpoints/online/model-1/sample-request.json"
ml_client.online_endpoints.invoke(
    endpoint_name=sai_endpoint_name, 
    request_file=sample_data)

### User-assigned managed identity

In [ ]:
sample_data = "endpoints/online/model-1/sample-request.json"
ml_client.online_endpoints.invoke(
    endpoint_name=uai_endpoint_name, 
    request_file=sample_data)

## Delete the endpoint and storage account

### System-assigned managed identity

In [ ]:
ml_client.online_endpoints.begin_delete(sai_endpoint_name)

In [ ]:
storage_client.storage_accounts.delete(
    resource_group_name=resource_group,
    account_name=storage_account_name)

### User-assigned managed identity

In [ ]:
ml_client.online_endpoints.begin_delete(uai_endpoint_name)

In [ ]:
storage_client.storage_accounts.delete(
    resource_group_name=resource_group,
    account_name=storage_account_name)